In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

from scipy.ndimage import shift
import matplotlib.pyplot as plt
import keras.api._v2.keras as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout, Input, BatchNormalization
from keras.optimizers.legacy import SGD


We will create a MineSweeper game with a board size of (a,a). The game will be implemented using two matrix of size (a,a), which will be named the Game board and the Mine board. 

The Game board will be a matrix where: -1 represent an covered cell, "@" represent an uncovered cell without any mine in its surrounding, numbers 1-8 represent a cell with the number of bombs surrounding it and "*" represent a mine. It will be the game board observed by the player.

The Mine Board will be a matrix with 0 and 1, where 1 is a bomb and 0 is a safe cell. This matrix will store the mine location on the board.

In [17]:
class minesweeper_game(object):
    def __init__(self, coord = (0,0), len_board = 8, print_board = True):
        self.print_board = print_board
        self.len_board = len_board
        self.number_mines = self.len_board
        if print_board == True: 
            print("Welcome to MineSweeper!!!")
            print("The following board is your game board. It has {} mines".format(self.number_mines))
            print(np.full((self.len_board,self.len_board), ".", dtype=np.dtype('U100')))
            print("Coordinates ({},{}) revealed:".format(coord[0],coord[1]))
        self.mine_board, self.game_board = self.create_boards(coord)
        self.update_game_board(coord)
        self.uncover_all_empty_spaces()
        if print_board == True:
            print(self.game_board)
        
    def create_boards(self, coord): #We create both boards. We ask for coord so that the first input can not be a loss
        mine_board = np.full((self.len_board,self.len_board),0)  #We define a np.array of size (a,a) with only zeroes
        mines = [] #list of mines
        while len(mines) <= self.number_mines: #we want #self.len_board mines
            mine_coord = (random.randint(0, self.len_board-1), random.randint(0, self.len_board-1)) #we consider the coordinate of the mine
            if (coord != mine_coord) and (mine_coord not in mines): 
                mines.append(mine_coord)
                mine_board[mine_coord] = 1
        game_board = np.full((self.len_board,self.len_board),".") #game board is empty
        self.number_mines = len(mines)
        return mine_board, game_board
                    
    def game_status(self, coord): #Check the state of the game
        if self.mine_board[coord] == 1: #Uncovered mine = loss
            return 0
        uncovered_cells = self.len_board*self.len_board
        for i in range(0,self.len_board): #This for can be simplified using for iy, ix in np.ndindex(a.shape) to loop in all elements
            for j in range(0, self.len_board):
                if (self.game_board[(i,j)] != "."):
                    uncovered_cells -= 1
        if uncovered_cells == self.number_mines:
            return 1 #You uncovered all of the cells without a bomb (There are "a" bombs).
        else:
            return 2 #The game is still going
    
    def update_game_board(self, coord): #After the player gives a coordinates, update the map with the new info
        i = coord[0]
        j = coord[1]
        if self.mine_board[coord] == 1: #Bomb
            self.game_board[coord] = "*"
        else:
            count_mines = 0
            #We have to separate the cases on which (a,b) it is in the border because we have to add the surrounding number of bombs
            if i == 0: #First row 
                for m in range(0, 2): #In the future: This for can be simplified using for iy, ix in np.ndindex(a.shape) to loop in all elements.
                    if j == 0: #Corner (0,0)
                        for n in range(0, 2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Corner (0,len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = 0 without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            elif i == self.len_board-1: #Last row
                for m in range(i-1, i+1):
                    if j == 0: #corner (len, 0)
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #corner (len, len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = len without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            else: #Not in first or last row
                for m in range(i-1, i+2):
                    if j == 0: #First column
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Last column
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else:  #points is in the middle
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            if count_mines == 0: #No mines = empty
                self.game_board[coord] = "@"
            else:
                self.game_board[coord] = count_mines #Number of mines surrounding it

    def uncover_all_empty_spaces(self): #If the coordinate added was a "@", we clear all "@" and number surrounding it.
        #We want to consider only the cells which are above, below and in the sides of a "@".
        covered_empty = True
        local_game_board = np.copy(self.game_board) #Copy of the original board, to compare
        while covered_empty == True: #In the future: This loop can be simplified using for iy, ix in np.ndindex(a.shape) to loop in all elements.
            for i in range(0,self.len_board): #Rows
                for j in range(0,self.len_board): #Columns
                    if self.game_board[(i,j)] == "@":
                        if i == 0:
                            for m in range(0, 2): #we can only take row 0 and 1
                                if j == 0: #Corner (0,0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,1))): 
                                            self.update_game_board((m,n))
                                elif (j == self.len_board-1): #Corner (0,len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #row x = 0 without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,j-1)) and ( (m,n) != (1,j+1))):
                                            self.update_game_board((m,n))
                        elif i == self.len_board-1:
                            for m in range(self.len_board-2, self.len_board):
                                if j == 0: #corner (len, 0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, 1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #corner (len, len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, self.len_board-2) )):
                                            self.update_game_board((m,n))
                                else: #row x = len without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1)) and ( (m,n) != (i-1,j+1))) :
                                            self.update_game_board((m,n))
                        else:
                            for m in range(i-1, i+2):
                                if j == 0: #First column
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1,1) ) and ((m,n) != (i+1,1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #Last column
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, self.len_board-2) ) and ( (m,n) != (i+1, self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #coordinate is in the middle
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1) ) and ((m,n) != (i-1, j+1) ) and ((m,n) != (i+1, j-1) ) and ((m,n) != (i+1, j+1) )):
                                            self.update_game_board((m,n))
                    else:
                        continue
                else:
                    continue
            if np.array_equal(local_game_board, self.game_board):
                #To stop we check that the last loop didnt added a new cleared element.
                covered_empty = False
            else:
                local_game_board = np.copy(self.game_board) #If we added new cells, we update the local_game_board to compare again after the loop.
                
                
    def enter_coord(self, coord): #Method to reveal a new cell given a coordinate.
        i = coord[0]
        j = coord[1]
        i = int(i)
        j = int(j)
        coord = (i,j)
        if self.print_board == True:
            print("Coordinates ({},{}) revealed:".format(i,j))
        self.update_game_board(coord)
        self.uncover_all_empty_spaces()
        if self.print_board == True:
            print(self.game_board)
        game_status = self.game_status(coord)
        return game_status, self.game_board

    def check_if_loses(self, game_status): #Method just to print the status of the game.
        if game_status == 0:
            print("You lost!. You found a mine.")
        elif game_status == 1:
            print("You won!. You uncovered all the mines.")
        else:
            print("There are still mines left.")

#Helpful functions:
def input_int(): #To check that the input is the desire type
    Not_integer = True
    while Not_integer == True:
        try:
            len_board = int(input("Enter a number: "))
            return int(len_board)
        except ValueError:
            print("Input is not an integer")
    
def input_coord(): #To check that the input is the desire type
    Not_tuple = True
    while Not_tuple == True:
        try:
            coord = input("Enter a tuple of the form a,b: ").split(",")
            i = coord[0]
            j = coord[1]
            i = int(i)
            j = int(j)
            return (i,j)
        except ValueError:
            print("Input is not a tuple of two integers in the form a,b")


The MineSweeper works in the following way:

First, we create a game with
    game = minesweeper_game(len_board, coord)
The len_board is the length of the arrows (or columns) of the matrix, while the coord is the coordinate of the first revealed cell to generate the map.

Then, we start revealing more coordinates with
    game.enter_coord((a,b))
This method accept a tuple and returns the updated board game, and the game_status to check if it has won, lost or is still going.

In [18]:
game = minesweeper_game(len_board = 8, coord = (5,1))
game_status, game_board = game.enter_coord((0,0))
game.check_if_loses(game_status)


Welcome to MineSweeper!!!
The following board is your game board. It has 8 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (5,1) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '1' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '1' '.' '.' '.' '.' '.' '.']
 ['1' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '1' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
There are still mine

Now, we want to create a program, which will have a Neural Networks on its core, to learn how to play minesweeper. For this we will construct the following program:

1. A Possible_moves function, which given a board_game will give a List of every coordinate of each covered cell.
2. An Evaluator, that will be a Neural Network, which given a board_game with the dictionary of coordinates of covered cell, will evaluate each member of the dictionary and asigning a value between 0 and 1.
3. A Coordinate_selector, which will select the coordinate with the highest value and will apply it.

We start with the Possible_moves function.

In [19]:
def possible_moves(game_board):
    len_board = game_board.shape[0]
    possible_moves_list = []
    for i in range(0,len_board): #Rows
        for j in range(0,len_board): #Columns
            if game_board[(i,j)] == ".":
                possible_moves_list.append((i,j))
    return possible_moves_list #List of every non checked cell.

In [20]:
game = minesweeper_game(len_board = 8, coord = (5,1))
game_status, game_board = game.enter_coord((0,0))
possible_moves_list = possible_moves(game_board)
print(possible_moves_list) #It works.

Welcome to MineSweeper!!!
The following board is your game board. It has 8 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (5,1) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '2' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '@' '@' '@' '@' '@' '@' '@']
 ['1' '1' '1' '1' '1' '@' '@' '@']
 ['.' '.' '.' '.' '.' '1' '2' '1']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '2' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
[(2, 0), (2, 1), (2,

We follow with the Evaluator:

It will be a Neural Network which will asign a value to every element of board+possible_moves_list.

In [21]:
def NNmodel(len):
    model = Sequential()
    model.add(Dense(64,kernel_initializer='normal', activation='relu', input_shape=(len*len+2,)))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(128, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal', activation = "sigmoid"))
    learning_rate = 0.001
    momentum = 0.8
    sgd = SGD(learning_rate=learning_rate, momentum=momentum,nesterov=False)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

Then we combine the previous two function on a Coordinate_selector

In [22]:
def board_to_numbers(game_board):
    board_numbers = game_board.copy()
    board_numbers = board_numbers.astype(dtype='<U2') #First we change to dtype "U2" to convert "." to -1
    board_numbers[board_numbers == "."] = -1
    board_numbers[board_numbers == "@"] = 0
    board_numbers[board_numbers == "*"] = 9
    board_numbers = board_numbers.astype(dtype='int') #Then, when every element is an integer, we transform to dtype int
    return board_numbers #Numerical board to give to the neural network

def coordinate_selector(model, game_board): #Grab the board, compute possible moves and asign to each move a value
    values = {}
    len_board = game_board.shape[0]
    game_board_num = board_to_numbers(game_board)
    possible_moves_list = possible_moves(game_board)
    for move in possible_moves_list: #For every possible move
        play = np.append(game_board_num.reshape(len_board*len_board,-1), move).reshape(1,len_board*len_board+2) #The input will be (board size+move)
        score = model.predict(play, verbose = 0)
        values[move] = score #We save the move+score
    selected_coord = max(values, key=values.get) #We pick the one with the biggest value
    score = values[selected_coord]
    game_board_pick = np.append(game_board_num.reshape(len_board*len_board,-1), selected_coord).reshape(1,len_board*len_board+2)
    return game_board_pick, selected_coord, score

We are ready to test our program. It should return a selected_coord which we will input in our enter_coord method, and a score.

In [23]:
game = minesweeper_game(len_board = 8, coord = (2,3))
game_status, game_board = game.enter_coord((0,0))
model = NNmodel(game.len_board)
game_board_num, selected_coord, score = coordinate_selector(model, game_board)
print("Selected coord: ",selected_coord)
print("Score: ", score)
game_status, game_board = game.enter_coord(selected_coord)

Welcome to MineSweeper!!!
The following board is your game board. It has 8 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (2,3) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '2' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '@' '@' '1' '.' '.' '.' '.']
 ['@' '1' '1' '.' '.' '.' '.' '.']
 ['@' '2' '.' '2' '.' '.' '.' '.']
 ['@' '2' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']]
Selected coord:  (0,

We are ready to implement the self-learning algorithm.

In [24]:
def train(model, len_board, print_progress = False):
    if print_progress==True:
        print("___________________________________________________________________")
        print("Starting a new game")
    first_coord = (random.randint(0, len_board-1), random.randint(0, len_board-1))
    game = minesweeper_game(len_board = len_board, coord = first_coord)
    game_status = 2
    scores_list=[]
    corrected_scores_list = []
    new_board_states_list=[]
    number_of_turns = 0
    while game_status == 2:
        game_board_num, selected_coord, score = coordinate_selector(model, game.game_board)
        game_status, game.game_board = game.enter_coord(selected_coord)
        scores_list.append(score[0][0])
        new_board_states_list.append(game_board_num)
        number_of_turns += 1
        
    new_board_states_list=np.vstack(new_board_states_list)
    if game_status == 0:
        corrected_scores_list = shift(scores_list, -1, cval =-1.0)
        result = "Loss"
    if game_status == 1:
        corrected_scores_list = shift(scores_list, -1, cval =1.0)
        result = "Won"
    if print_progress==True:
        print("Program has ",result)
        print("\n Correcting the Scores and Updating the model weights")
        print("___________________________________________________________________\n")
    x=new_board_states_list
    y=corrected_scores_list 
    
    def unison_shuffled_copies(a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        return a[p], b[p]
    
    x,y = unison_shuffled_copies(x,y)
    x=x.reshape(-1,len_board*len_board+2) 
    
    model.fit(np.array(x),np.array(y),epochs=1,batch_size=1,verbose=0)
    return model,y,result
    

In [25]:
updated_model,y,result=train(model, len_board = 8, print_progress=True)

___________________________________________________________________
Starting a new game
Welcome to MineSweeper!!!
The following board is your game board. It has 8 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (1,0) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['1' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '@' '@' '@' '@' '1' '.' '.']
 ['1' '1' '@' '1' '1' '.' '.' '.']
 ['.' '1' '@' '1' '.' '.' '.' '.']
 ['1' '1' '@' '1' '.' '.' '.' '.']
 ['@' '@' '@' '@' '2' '.' '.' '.']
 ['@' '@' '@' '@' '3' '.' '.' '.']
 ['

In [26]:
game_counter=1
data_for_graph=pd.DataFrame()

while(game_counter<=1):
    updated_model,y,result=train(model, len_board = 8, print_progress=True)
    data_for_graph=pd.concat([data_for_graph, pd.Series({"game_counter":game_counter,"result":result})])
    if game_counter % 10000 == 0:
        print("Game#: ",game_counter)
    print(game_counter)
    game_counter+=1


___________________________________________________________________
Starting a new game
Welcome to MineSweeper!!!
The following board is your game board. It has 8 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (7,0) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['1' '.' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '1' '.' '.' '.' '.' '.']
 ['@' '@' '@' '2' '.' '.' '.' '.']
 ['@' '@' '@' '1' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['*' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['1' '.' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '1' '.' '.' '.' '.' '.']
 ['

If we run the program long enough, we can conclude that introducing the full board will give too much noise. In reality, when we play minesweeper, we analyze locally to take a decision about if a coord is a mine or not. 

We will change both the coordinate selector and the neural network. We want to minimize the board introduced. Thus, we will define a local_board for each coordinate, that will return a 5x5 board with the coord selected in the center. For the coordinates in the border we will expand the boardsize adding two rows/columns of -2.

In [27]:
def extend_board(game_board):
    len_board = game_board.shape[0]
    board_num = board_to_numbers(game_board)
    extended_board = np.full((len_board+4,len_board+4), -2)
    for i in range(0,len_board): #Rows
        for j in range(0,len_board): #Columns
            extended_board[i+2,j+2] = board_num[i,j]
    return extended_board

extended_board = extend_board(game_board)
print("Game board:")
print(game_board)
print("Extended numerical board:")
print(extended_board)

def local_board(game_board, coord):
    extended_board = extend_board(game_board)
    local_board = np.full((5,5), -2)
    for i in range(0,5):
        for j in range(0,5):
            local_board[i,j] = extended_board[coord[0]+i, coord[1]+j]
    return local_board

print("Local board:") 
print(local_board(game_board, (1,1)))

def coordinate_selector_updated(model, game_board): 
    values = {}
    possible_moves_list = possible_moves(game_board)
    for move in possible_moves_list: 
        local_game_board = local_board(game_board, move)
        play = local_game_board.reshape(-1,25)
        score = model.predict(play, verbose = 0) 
        values[move] = score 
    selected_coord = max(values, key=values.get) 
    score = values[selected_coord]
    local_board_pick = local_board(game_board, selected_coord)
    return local_board_pick, selected_coord, score

Game board:
[['@' '@' '@' '1' '*' '.' '.' '.']
 ['@' '1' '1' '.' '.' '.' '.' '.']
 ['@' '2' '.' '2' '.' '.' '.' '.']
 ['@' '2' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']]
Extended numerical board:
[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2  0  0  0  1  9 -1 -1 -1 -2 -2]
 [-2 -2  0  1  1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  2 -1  2 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  2 -1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  1 -1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  0  1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  0  1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2  0  0  1 -1 -1 -1 -1 -1 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]]
Local board:
[[-2 -2 -2 -2 -2]
 [-2  0  0  0  1]
 [-2  0  1  1 -1]
 [-2  0  2 -1  2]
 [-2  0  2 -1 -1]]


The new NN model will be:

In [28]:
def NNmodel_updated():
    model = Sequential()
    model.add(BatchNormalization(input_shape=[25,])),
    model.add(Dense(8,kernel_initializer='normal', activation='relu', input_shape=(25,)))
    model.add(Dense(16, kernel_initializer='normal',activation='relu'))
    model.add(Dense(16, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal', activation = "sigmoid"))
    learning_rate = 0.001
    momentum = 0.8
    sgd = SGD(learning_rate=learning_rate, momentum=momentum,nesterov=False)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model


def train_updated(model, len_board, print_board = True, print_progress = False):
    if print_progress==True:
        print("___________________________________________________________________")
        print("Starting a new game")
    first_coord = (random.randint(0, len_board-1), random.randint(0, len_board-1))
    game = minesweeper_game(len_board = len_board, coord = first_coord, print_board = print_board)
    game_status = 2
    scores_list=[]
    new_board_states_list=[]
    number_of_turns = 0
    while game_status == 2:
        local_board_num, selected_coord, score = coordinate_selector_updated(model, game.game_board)
        game_status, game.game_board = game.enter_coord(selected_coord)
        if game_status == 0:
            final_score = score[0][0]
            score[0][0] = int(-1)
        else:
            score[0][0] = int(1)
        scores_list.append(score[0][0])
        new_board_states_list.append(local_board_num.reshape(-1,25))
        number_of_turns += 1
    final_board = game.game_board
    new_board_states_list=np.vstack(new_board_states_list)
    if print_progress==True:
        print("Program has ",result)
        print("\n Correcting the Scores and Updating the model weights")
        print("___________________________________________________________________\n")
    x=new_board_states_list
    y= np.array(scores_list)

    def unison_shuffled_copies(a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        return a[p], b[p]
    
    x,y = unison_shuffled_copies(x,y) #Shuffling before fitting to remove some kind of pattern
    x=x.reshape(-1,25) 
    
    model.fit(np.array(x),np.array(y),epochs=1,batch_size=1,verbose=0)
    return model, final_score, final_board, result

In [31]:
game_counter=0
data_for_graph=pd.DataFrame()
updated_model = NNmodel_updated()
number_of_wins = 0
print_progress = False
print_board = False
while(game_counter<=1):
    updated_model,final_score, final_game_board, result=train_updated(updated_model, len_board = 5, print_progress=print_progress, print_board = print_board)
    if result == "Won":
        number_of_wins += 1 
    #data_for_graph=pd.concat([data_for_graph, pd.Series({"game_counter":game_counter,"result":result})])
    if game_counter % 1 == 0:
        print("Game number: ",game_counter, "Game won: ", number_of_wins)
        print("Score of last play: ", final_score)
        print(final_game_board)
    game_counter+=1
    print_progress = False
    print_board = False
print(number_of_wins)

Game number:  0 Game won:  0
Score of last play:  0.5005529
[['.' '.' '.' '*' '.']
 ['.' '.' '.' '3' '1']
 ['.' '.' '.' '1' '@']
 ['1' '1' '1' '@' '@']
 ['@' '@' '@' '@' '@']]
Game number:  1 Game won:  0
Score of last play:  0.50011516
[['.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '*']
 ['.' '.' '.' '.' '1']
 ['1' '.' '.' '1' '@']
 ['@' '1' '.' '1' '@']]
0
